In [2]:
from __future__ import print_function

from collections import defaultdict

from intervaltree import IntervalTree, Interval

In [3]:
def overlap_intervals(self, other):
    splits = (self | other)
    splits.split_overlaps()
    self_int_other = IntervalTree(filter(lambda r: self.overlaps(r) and other.overlaps(r), splits))
    return self_int_other


def overlap_all(trees):
    if len(trees) < 2:
        return None

    overlap = overlap_intervals(trees[0], trees[1])
    for t in trees[2:]:
        overlap = overlap_intervals(overlap, t)
    overlap.merge_overlaps(data_reducer)

    return overlap


def data_reducer(current_data, new_data):
    return current_data | new_data

In [4]:
gtrees = defaultdict(dict)
all_genes = set()
files = defaultdict(lambda: defaultdict(list))
#allref= defaultdict(lambda: defaultdict(list))

In [5]:
for f in ('vqc', 'iqc', 'sqc', 'mqc'):
    for line in open(f): 
            x = line.rstrip().split(' ')
            if  "Real" in  str(x[0]):
                files[f][x[14]].append((int(x[3]), int(x[4])))
print('done')

done


In [13]:
gtrees = defaultdict(dict)
unaligned_regions = defaultdict(dict)
#unaligned_regions = defaultdict(dict)
genome_size = 10000 #2056037
unaligned = IntervalTree()
unaligned[1:genome_size] = set()
all_genes = set()
for f in files:
    for g in files[f]:
        all_genes.add(g)
        gtrees[g][f] = IntervalTree(Interval(iv[0], iv[1], set([f]))
                                    for iv in files[f][g])
        for interval in gtrees[g][f]:
                unaligned=gtrees[g][f].chop(interval.begin(), interval.end()) 
        unaligned_regions=unaligned[g][f]
print('done')   

TypeError: 'int' object is not callable

In [11]:
overlap_trees = {}
for gene in all_genes:
    overlap_trees[gene] = overlap_all(list(unaligned[gene].values())) #just getting the overlaps of the aligned  chopped reads
    if overlap_trees[gene]:
        print('@', gene, end=": \n ")
        print(overlap_trees[gene])  
print('done')

AttributeError: 'set' object has no attribute 'values'

In [ ]:
print(overlap_trees['Desulfovibrio_vulgaris_DP4'].begin())
print(overlap_trees['Desulfovibrio_vulgaris_DP4'].end())
print ('done')

In [ ]:
cutoff =10
out =open('uncovered','w+')
overlap_trees = {}
for gene in all_genes:
    overlap_trees[gene] = overlap_all(list(gtrees[gene].values()))
    if overlap_trees[gene]:
        print('@',gene,":",overlap_trees[gene].begin( ),",",overlap_trees[gene].end(), len(overlap_trees[gene])) 
        if len(overlap_trees[gene]) >cutoff:
                for i in overlap_trees[gene]:
                        print(gene,',',i,',',file=out)
                    
print ('done')       

In [ ]:
#parse case files
gdict=defaultdict(list)
input=open('uncovered','r')
for line in input:
        line=line.split(',')
        #print(line)
        gene=line[0]
        x=line[1].split('(')
        x=x[1]
        y=line[2]
        #print(x,"-",y)
        gdict[line[0]].append(x)
        gdict[line[0]].append(y)


for g in gdict.keys(): 
         i=1
         l=len(gdict[g])
         min =20700000
         max =-1
         sum=0
         while(i <l-1) :
                gap=abs( int(gdict[g][i+1]) -int(gdict[g][i]) )
                sum+=gap
                if min >gap: 
                        min=gap
                if max <gap:
                        max=gap
                i=i+2
         print(g,"has ",l/2,"misassemblies with min/max/average gaps:",str(min), "/",str(max),"/",(sum/l))